We are going to be using the following parameters:

$S_0 = 80$

$K = 80$

$ r = 0.055$

$\sigma = 0.35$

$T = 0.25$

# Team Member A - Atakan Devrent

For the Heston model, I am going to use the following parameters:

$\nu_0 = 0.032$

$\kappa_{\nu} = 1.85 $

$ \theta{\nu} = 0.045 $

First, we remind ourselves the **Heston** model for option pricing which uses stochastic volatility:

$$
\begin{align*}
dS_t &= \mu S_t dt + \sqrt{v_t} S_t dW^S_t \\
dv_t &= \kappa(\theta - v_t) dt + \sigma \sqrt{v_t} dW^v_t \\
dW^S_t dW^v_t &= \rho dt
\end{align*}
$$

Where:
$$
\begin{align*}
S_t &: \text{Asset price at time } t \\
v_t &: \text{Variance of the asset price at time } t \\
\mu &: \text{Drift of the asset price} \\
\kappa &: \text{Rate of mean reversion} \\
\theta &: \text{Long-term mean of the variance} \\
\sigma &: \text{Volatility of volatility} \\
W^S_t, W^v_t &: \text{Wiener processes} \\
\rho &: \text{Correlation between the two Wiener processes}
\end{align*}
$$

## 5, 6, 7)

In [12]:
# Team Member A

import numpy as np
from scipy.stats import norm

def heston_model_monte_carlo(S0, K, T, r, v0, kappa, theta, sigma, rho, num_simulations, num_steps):
    dt = T / num_steps
    
    S = np.zeros((num_simulations, num_steps + 1))
    v = np.zeros((num_simulations, num_steps + 1))
    
    S[:, 0] = S0
    v[:, 0] = v0
    
    z1 = np.random.normal(0, 1, (num_simulations, num_steps))
    z2 = rho * z1 + np.sqrt(1 - rho**2) * np.random.normal(0, 1, (num_simulations, num_steps))
    
    for t in range(1, num_steps + 1):
        S[:, t] = S[:, t-1] * np.exp((r - 0.5 * v[:, t-1]) * dt + np.sqrt(v[:, t-1] * dt) * z1[:, t-1])
        v[:, t] = np.maximum(v[:, t-1] + kappa * (theta - v[:, t-1]) * dt + sigma * np.sqrt(v[:, t-1] * dt) * z2[:, t-1], 0)
    
    call_payoffs = np.maximum(S[:, -1] - K, 0)
    put_payoffs = np.maximum(K - S[:, -1], 0)
    
    call_price = np.exp(-r * T) * np.mean(call_payoffs)
    put_price = np.exp(-r * T) * np.mean(put_payoffs)
    
    return call_price, put_price

def calculate_greeks(S0, K, T, r, v0, kappa, theta, sigma, rho, num_simulations, num_steps):
    call_price, put_price = heston_model_monte_carlo(S0, K, T, r, v0, kappa, theta, sigma, rho, num_simulations, num_steps)
    
    dS = 0.01 * S0
    call_price_up, put_price_up = heston_model_monte_carlo(S0 + dS, K, T, r, v0, kappa, theta, sigma, rho, num_simulations, num_steps)
    call_price_down, put_price_down = heston_model_monte_carlo(S0 - dS, K, T, r, v0, kappa, theta, sigma, rho, num_simulations, num_steps)
    
    call_delta = (call_price_up - call_price_down) / (2 * dS)
    put_delta = (put_price_up - put_price_down) / (2 * dS)
    
    call_gamma = (call_price_up - 2*call_price + call_price_down) / (dS**2)
    put_gamma = (put_price_up - 2*put_price + put_price_down) / (dS**2)
    
    return call_price, put_price, call_delta, put_delta, call_gamma, put_gamma

# Parameters
S0 = 80  # Initial stock price
K = 80   # Strike price (ATM)
T = 3/12  # Time to maturity in years
r = 0.055  # Risk-free rate
v0 = 0.032  # Initial variance
kappa = 1.85  # Mean reversion speed
theta = 0.045  # Long-term variance
sigma_v = 0.35  # Volatility of variance
num_simulations = 100000
num_steps = 100

# Question 5: rho = -0.30
rho_5 = -0.30
results_5 = calculate_greeks(S0, K, T, r, v0, kappa, theta, sigma_v, rho_5, num_simulations, num_steps)

# Question 6: rho = -0.70
rho_6 = -0.70
results_6 = calculate_greeks(S0, K, T, r, v0, kappa, theta, sigma_v, rho_6, num_simulations, num_steps)

# Print results
print("Question 5 Results (rho = -0.30):")
print(f"ATM European Call Price: {results_5[0]:.2f}")
print(f"ATM European Put Price: {results_5[1]:.2f}")

print("\nQuestion 6 Results (rho = -0.70):")
print(f"ATM European Call Price: {results_6[0]:.2f}")
print(f"ATM European Put Price: {results_6[1]:.2f}")

print("\nQuestion 7 Results:")
print("For rho = -0.30:")
print(f"Call Delta: {results_5[2]:.2f}")
print(f"Put Delta: {results_5[3]:.2f}")
print(f"Call Gamma: {results_5[4]:.2f}")
print(f"Put Gamma: {results_5[5]:.2f}")

print("\nFor rho = -0.70:")
print(f"Call Delta: {results_6[2]:.2f}")
print(f"Put Delta: {results_6[3]:.2f}")
print(f"Call Gamma: {results_6[4]:.2f}")
print(f"Put Gamma: {results_6[5]:.2f}")

Question 5 Results (rho = -0.30):
ATM European Call Price: 3.46
ATM European Put Price: 2.38

Question 6 Results (rho = -0.70):
ATM European Call Price: 3.51
ATM European Put Price: 2.41

Question 7 Results:
For rho = -0.30:
Call Delta: 0.61
Put Delta: -0.40
Call Gamma: 0.04
Put Gamma: 0.05

For rho = -0.70:
Call Delta: 0.62
Put Delta: -0.36
Call Gamma: -0.02
Put Gamma: 0.01
